**R_network_analysis_test.R.ipynb** - play with R and large matrices, see how it goes.

# R network analysis files

Related files:

* network descriptives
    * network-level
        
        * files
            
            * R scripts:
                
                * `context_analysis/R/db_connect.r`
                * `context_analysis/R/sna/functions-sna.r`
                * `context_analysis/R/sna/sna-load_data.r`
                * `context_analysis/R/sna/igraph/*`
                * `context_analysis/R/sna/statnet/*`
        
        * statnet/sna
            
            * `sna::gden()` - graph density
            * R scripts:
                
                * `context_analysis/R/sna/statnet/sna-statnet-init.r`
                * `context_analysis/R/sna/statnet/sna-statnet-network-stats.r`
                * `context_analysis/R/sna/statnet/sna-qap.r`
        
        * igraph
            
            * `igraph::transitivity()` - vector of transitivity scores for each node in a graph, plus network-level transitivity score.
                
                * Q - interpretation?
                
            * R scripts:
                
                * `context_analysis/R/sna/statnet/sna-igraph-init.r`
                * `context_analysis/R/sna/statnet/sna-igraph-network-stats.r`


# Setup

## Setup -  working directories

Store important directories and file names in variables:

In [ ]:
getwd()

In [ ]:
# code files (in particular SNA function library, modest though it may be)
base_code_directory <- "/home/jonathanmorgan/work/django/research/research/context_analysis/R"
sna_code_directory <- paste( base_code_directory, "/sna", sep = "" )
code_directory <- sna_code_directory

# home directory
home_directory <- getwd()
home_directory <- "/home/jonathanmorgan/work/django/research/research/work/phd_work/analysis/network_data"

# data directories
#data_directory <- paste( home_directory, "/data", sep = "" )
data_directory <- "/home/jonathanmorgan/shares/phd_work/network_data/grp_6mos_by_month-take1"
workspace_file_name <- "statnet-grp-test.RData"
workspace_file_path <- paste( data_directory, "/", workspace_file_name )

In [ ]:
# set working directory to data directory for now.
setwd( data_directory )
getwd()

## Setup - functions

### Setup - functions - import SNA functions

source the file `functions-sna.r`.

In [ ]:
sna_function_file_path <- paste( sna_code_directory, "/", 'functions-sna.r', sep = "" )

In [ ]:
source( sna_function_file_path )

### Setup - functions - import statnet functions

- Back to [Table of Contents](#Table-of-Contents)

source the file `statnet/functions-statnet.r`.  This file includes:

- `compareMatricesQAP()`, used extensively below.

In [ ]:
# statnet/sna functions
# - /home/jonathanmorgan/work/django/research/context_analysis/R/sna/stanet/functions-statnet.r
statnetFunctionFilePath <- paste( sna_code_directory, "/statnet/", 'functions-statnet.r', sep = "" )

In [ ]:
source( statnetFunctionFilePath )

## Setup - ReferenceClasses

### Setup - ReferenceClass - NetworkInfo

In [ ]:
networkInfoClassFilePath <- paste( sna_code_directory, "/NetworkInfo.r", sep = "" )

In [ ]:
source( networkInfoClassFilePath )

### Setup - ReferenceClass - NetworkBeforeAfterDatePair

In [ ]:
# ReferenceClass NetworkBeforeAfterDatePair
# - for help: `help( ReferenceClasses )`
# - https://www.rdocumentation.org/packages/methods/versions/3.6.2/topics/ReferenceClasses

# Dependencies:
# - Depends on NetworkInfo ReferenceClass.
# - Depends on functions-statnet.r

# Usage:
# testNetworkPair <- NetworkBeforeAfterDatePair()
# testNetworkPair$myBaseDate <<- "2009-05-12"
# testNetworkPair$myLabel <<- paste( testNetworkPair$myBaseDate, "-6mos", sep = "" )


#==============================================================================#
# imports
#==============================================================================#

#==============================================================================#
# class definition
#==============================================================================#

networkBeforeAfterDatePair_fields <- list(
    myAfterNetwork = "ANY",
    myAfterNetworkMatrix = "matrix",
    myBaseDate = "character",
    myBeforeNetwork = "ANY",
    myBeforeNetworkMatrix = "matrix",
    myGraphCorrelation = "numeric",
    myGraphCorrelationQAPResult = "ANY",
    myGraphCovariance = "numeric",
    myGraphCovarianceQAPResult = "ANY",
    myGraphHammingDistance = "numeric",
    myGraphHammingDistanceQAPResult = "ANY",
    myLabel = "character",
    myMatrixComparison = "ANY"
)

NetworkBeforeAfterDatePair <- setRefClass(
    "NetworkBeforeAfterDatePair",
    fields = networkBeforeAfterDatePair_fields
)

#==============================================================================#
# instance methods
#==============================================================================#

NetworkBeforeAfterDatePair$methods(
    compareMatrices = function( outputPlotsIN = FALSE,
                                debugFlagIN = FALSE,
                                repsIN = 10,
                                doQapIN = FALSE ) {

        # declare variables
        outputPrefix <- NULL
        beforeMatrix <- NULL
        afterMatrix <- NULL
        
        # set output prefix to the file name.
        outputPrefix <- myLabel
        
        # retrieve matrices
        beforeMatrix <- myBeforeNetworkMatrix
        afterMatrix <- myAfterNetworkMatrix

        myMatrixComparison <<- compareMatricesQAP(
            beforeMatrix,
            afterMatrix,
            outputPrefixIN = outputPrefix,
            outputPlotsIN = outputPlotsIN,
            debugFlagIN = debugFlagIN,
            repsIN = repsIN,
            doQapIN = doQapIN )
        
        # retrieve and store individual values.
        myGraphCorrelation <<- myMatrixComparison$graphCorrelation
        myGraphCorrelationQAPResult <<- myMatrixComparison$qapGcorResult
        myGraphCovariance <<- myMatrixComparison$graphCovariance
        myGraphCovarianceQAPResult <<- myMatrixComparison$qapGcovResult
        myGraphHammingDistance <<- myMatrixComparison$graphHammingDist
        myGraphHammingDistanceQAPResult <<- myMatrixComparison$qapHdistResult

        if ( doQapIN == TRUE ){
            
            # also output plots of distributions of QAP values?
            if ( outputPlotsIN == TRUE ){
                
                # yes, output those plots, also, I guess.
                displayCompareMatricesQAPOutput( myMatrixComparisonData, outputPrefix, TRUE )
            
            }
            
        }
        
    } #-- END method compareMatrices() --#
    
)

NetworkBeforeAfterDatePair$methods(
    loadAfterNetwork = function( dataDirPathIN, fileNameIN ) {
        
        'Accepts data directory and file name, combines them into path,
         loads the network, then processes it so we are ready to compare.'

        # declare variables
        workNetwork <- NULL
        
        # create NetworkInfo instance
        workNetwork <- NetworkInfo()
        
        # init using file information passed in.
        workNetwork$initFromTabData( dataDirPathIN, fileNameIN )
        
        # process the network.
        workNetwork$processNetwork()
        
        # store matrix
        myAfterNetworkMatrix <<- workNetwork$myNetworkMatrix
        
        # store inside
        myAfterNetwork <<- workNetwork
        
    } #-- END method loadAfterNetwork() --#
)

NetworkBeforeAfterDatePair$methods(
    loadBeforeNetwork = function( dataDirPathIN, fileNameIN ) {
        
        'Accepts data directory and file name, combines them into path,
         loads them into data frame, then makes all the other objects
         we need.'

        # declare variables
        workNetwork <- NULL
        
        # create NetworkInfo instance
        workNetwork <- NetworkInfo()
        
        # init using file information passed in.
        workNetwork$initFromTabData( dataDirPathIN, fileNameIN )
        
        # process the network.
        workNetwork$processNetwork()
        
        # store matrix
        myBeforeNetworkMatrix <<- workNetwork$myNetworkMatrix
        
        # store inside
        myBeforeNetwork <<- workNetwork
        
    } #-- END method loadBeforeNetwork() --#
)

message( paste( "ReferenceClass NetworkBeforeAfterDatePair defined @ ", date(), sep = "" ) )


## Setup - network data

### Setup - network data - notes: render and store network data

First, need render to render network data and upload it to your server.

Directions for creating network data: 

- network data creation example notebook: [analysis-network_data_output_example.ipynb](../analysis-network_data_output_example.ipynb)

    - this shows programmatic data creation using JSON file to capture request parameters.
    - includes all parameters and a few examples of different ways you can set parameters.

- actual network data creation notebook: [analysis-network_data_output-GRP.ipynb](./analysis-network_data_output-GRP.ipynb)

You want a tab-delimited matrix that includes both the network and attributes of nodes as columns, and you want it to include a header row.

Once you render your network data files, you should place them on the server.

High level data file layout:

- tab-delimited.
- first row and first column are labels
- last 2 columns are traits of nodes (`person_id` and `person_type`)
- each row and column after first until the trait columns represents a person found in one of the articles.
- The people are in the same order from top to bottom and left to right.
- Where the row and column of two people meet, and one of the people is an author, the nunber in the cell where they meet is the number of times the non-author was quoted in an article by the author.  Does not include more basic two-mode co-location ties (appeared in same article, even if not an author and/or not quoted).


# Actual processing using ReferenceClasses

## NetworkInfo - process single network

In [ ]:
testNetworkInfo <- NetworkInfo()
testNetworkInfo$initFromTabData(
    data_directory,
    "grp_6mos_by_month_2005-07-03_post-20220604-040716.tab"
)
testNetworkInfo$processNetwork()

In [ ]:
testNetworkInfo$myDegreeAverage

## NetworkInfo - process two single networks

In [ ]:
processBeforeAfterNetworks <- function( beforeDataDirectoryIN, beforeFileIN, afterDataDirectoryIN, afterFileIN ) {

    testBeforeNetworkInfo <- NetworkInfo()
    testBeforeNetworkInfo$initFromTabData(
        beforeDataDirectoryIN,
        beforeFileIN
    )
    testBeforeNetworkInfo$processNetwork()
    
    testAfterNetworkInfo <- NetworkInfo()
    testAfterNetworkInfo$initFromTabData(
        afterDataDirectoryIN,
        afterFileIN
    )
    testAfterNetworkInfo$processNetwork()
    
    # get matrices from each and compare.
    testBeforeMatrix = testBeforeNetworkInfo$myNetworkMatrix
    testAfterMatrix = testAfterNetworkInfo$myNetworkMatrix

    # other variables
    outputPrefix <- paste( "2005-07-03", "-6mos", sep = "" )

    # call compare method
    myMatrixComparison <- compareMatricesQAP(
        testBeforeMatrix,
        testAfterMatrix,
        outputPrefixIN = outputPrefix,
        outputPlotsIN = FALSE,
        debugFlagIN = FALSE,
        repsIN = 10,
        doQapIN = FALSE )

    # retrieve and store individual values.
    myGraphCorrelation <- myMatrixComparison$graphCorrelation
    myGraphCorrelationQAPResult <- myMatrixComparison$qapGcorResult
    myGraphCovariance <- myMatrixComparison$graphCovariance
    myGraphCovarianceQAPResult <- myMatrixComparison$qapGcovResult
    myGraphHammingDistance <- myMatrixComparison$graphHammingDist
    myGraphHammingDistanceQAPResult <- myMatrixComparison$qapHdistResult
    
} #-- END function processBeforeAfterNetworks() --#


In [ ]:
processBeforeAfterNetworks(
    data_directory,
    "grp_6mos_by_month_2005-07-03_pre-20220604-040520.tab",
    data_directory,
    "grp_6mos_by_month_2005-07-03_post-20220604-040716.tab"
)

## NetworkBeforeAfterDatePair - process pair of networks

In [ ]:
# Create pair instance and set basic information
testNetworkPair <- NetworkBeforeAfterDatePair()
testNetworkPair$myBaseDate <- "2005-07-03"
testNetworkPair$myLabel <- paste( testNetworkPair$myBaseDate, "-6mos", sep = "" )


In [ ]:
testNetworkPair$loadBeforeNetwork(
    data_directory,
    "grp_6mos_by_month_2005-07-03_pre-20220604-040520.tab"
)

- average degree = 1.37457344078995
- average author degree (2 and 4) = 60.1761006289308
- average author degree (only 2) = 62.6953642384106
- average source degree (3 and 4) = 1.26808681672026
- average source degree (only 3) = 1.25590128755365
- degree SD = 11.2888518966015
- degree variance = 127.438177143404
- degree max = 321
- degree centrality = 0.0232117228168916
- betweenness centrality = 0.036634060879916
- connectedness = 0.294067754201938
- Warning message in sna::gtrans(myNetworkStatnet, mode = "graph"):
“gtrans called with use.adjacency=TRUE, but your data looks too large for that to work well.  Overriding to edgelist method.”
- transitivity = 0.00342872708081513
- density = 9.98092826597409e-05

In [ ]:
testNetworkPair <- NetworkBeforeAfterDatePair()
testNetworkPair$loadAfterNetwork(
    data_directory,
    "grp_6mos_by_month_2005-07-03_post-20220604-040716.tab"
)

- average degree = 1.38836854715748
- average author degree (2 and 4) = 58.6121212121212
- average author degree (only 2) = 59.171974522293
- average source degree (3 and 4) = 1.2935140113143
- average source degree (only 3) = 1.24469906492822
- degree SD = 11.095706272877
- degree variance = 123.114697693963
- degree max = 295
- degree centrality = 0.0213225583410718
- betweenness centrality = 0.0303777406610043
- connectedness = 0.303995646265527
- Warning message in sna::gtrans(myNetworkStatnet, mode = "graph"):
“gtrans called with use.adjacency=TRUE, but your data looks too large for that to work well.  Overriding to edgelist method.”
- transitivity = 0.00335767914962283
- density = 0.000100810960438388

In [ ]:
testNetworkPair$compareMatrices()

In [ ]:
testNetwork <- testNetworkPair$myBeforeNetwork
testNetwork$myDegreeAverage

In [ ]:
testNetworkPair$myGraphCorrelation

# garbage collection

In [ ]:
rm( testNetworkInfo )
rm( testNetworkPair )
gc()

# Workspace

## Workspace - load workspace (optional)

If you want, you can load this file's workspace, from a previous run:

In [ ]:
# assumes that you've already set working directory above to the
#     working directory.
setwd( data_directory )
load( workspace_file_name )

## Workspace - Save workspace image

- Back to [Table of Contents](#Table-of-Contents)

Save all the information in the current image, in case we need/want it later.

In [ ]:
# help( save.image )
message( paste( "Output workspace to: ", workspace_file_name, sep = "" ) )
save.image( file = workspace_file_name )

# Appendices

## Appendix - Example single network analysis

### `test` - loading large network matrix

First, we'll just try to load a 5 GB network matrix (all people from entire time, ties from articles in 2005).

In [ ]:
# initialize variables
testDataFolder <- data_directory
testDataFile <- "all_grp_hard_news_2005-20220603-015300.txt"
testDataPath <- paste( testDataFolder, "/", testDataFile, sep = "" )

In [ ]:
testDataPath

Load the data file into memory

In [ ]:
# tab-delimited:
testDataDF <- read.delim( testDataPath, header = TRUE, row.names = 1, check.names = FALSE )

In [ ]:
# get count of rows...
testRowCount <- nrow( testDataDF )
paste( "test row count = ", testRowCount, sep = "" )

# ...and columns
testColumnCount <- ncol( testDataDF )
paste( "test column count = ", testColumnCount, sep = "" )

Get just the tie rows and columns for initializing network libraries.

In [ ]:
# the below syntax returns only as many columns as there are rows, so
#     omitting any trait columns that lie in columns on the right side
#     of the file.
testNetworkDF <- testDataDF[ , 1 : testRowCount ]
#str( testNetworkDF )

In [ ]:
# convert to a matrix
testNetworkMatrix <- as.matrix( testNetworkDF )
# str( testNetworkMatrix )

### `test` - initialize statnet

- Back to [Table of Contents](#Table-of-Contents)

First, load the statnet package, then load the automated grp_month data into statnet object and assign attributes to nodes.

Based on `context_text/R/sna/statnet/sna-statnet-init.r`.

In [ ]:
# make sure you've loaded the statnet library
#install.packages( "statnet" )
library( statnet )

In [ ]:
# If you have a data frame of attributes (each attribute is a column, with
#     attribute name the column name), you can associate those attributes
#     when you create the network.
# attribute help: http://www.inside-r.org/packages/cran/network/docs/loading.attributes

# load attributes from a file:
#tab_attribute_test1 <- read.delim( "tab-test1-attribute_data.txt", header = TRUE, row.names = 1, check.names = FALSE )

# or create DataFrame by just grabbing the attribute columns
testNetworkAttributeDF <- testDataDF[ , ( testColumnCount - 1 ) : testColumnCount ]

# convert matrix to statnet network object instance.
testNetworkStatnet <- network( testNetworkMatrix, matrix.type = "adjacency", directed = FALSE, vertex.attr = testNetworkAttributeDF )

# look at information now.
testNetworkStatnet

# Network attributes:
#  vertices = 314
#  directed = FALSE
#  hyper = FALSE
#  loops = FALSE
#  multiple = FALSE
#  bipartite = FALSE
#  total edges= 309
#    missing edges= 0
#    non-missing edges= 309
#
# Vertex attribute names:
#    person_type vertex.names
#
# No edge attributes


### `test` - Basic metrics

- Back to [Table of Contents](#Table-of-Contents)

In [ ]:
# assuming that our statnet network object is in reference test1_statnet.

# Use the degree function in the sna package to create vector of degree values
#    for each node.  Make sure to pass the gmode parameter to tell it that the
#    graph is not directed (gmode = "graph", instead of "digraph").
# Doc: http://www.inside-r.org/packages/cran/sna/docs/degree
#degree_vector <- degree( test1_statnet, gmode = "graph" )

# If you have other libraries loaded that also implement a degree function, you
#    can also call this with package name:
testDegreeVector <- sna::degree( testNetworkStatnet, gmode = "graph" )

# output the vector
#testDegreeVector

# want more info on the degree function?  You can get to it eventually through
#    the following:
#help( package = "sna" )
#??sna::degree

# what is the average (mean) degree?
testAvgDegree <- mean( testDegreeVector )
paste( "average degree = ", testAvgDegree, sep = "" )

# subset vector to get only those that are above mean
testAboveMeanVector <- testDegreeVector[ testDegreeVector > testAvgDegree ]

# Take the degree and associate it with each node as a node attribute.
#    (%v% is a shortcut for the get.vertex.attribute command)
testNetworkStatnet %v% "degree" <- testDegreeVector

# also add degree vector to original data frame
testDataDF$degree <- testDegreeVector


In [ ]:
# average author degree (person types 2 and 4)
testAverageAuthorDegree2And4 <- calcAuthorMeanDegree( dataFrameIN = testDataDF, includeBothIN = TRUE )
paste( "average author degree (2 and 4) = ", testAverageAuthorDegree2And4, sep = "" )

# average author degree (person type 2 only)
testAverageAuthorDegreeOnly2 <- calcAuthorMeanDegree( dataFrameIN = testDataDF, includeBothIN = FALSE )
paste( "average author degree (only 2) = ", testAverageAuthorDegreeOnly2, sep = "" )

# average source degree (person types 3 and 4)
testAverageSourceDegree3And4 <- calcSourceMeanDegree( dataFrameIN = testDataDF, includeBothIN = TRUE )
paste( "average source degree (3 and 4) = ", testAverageSourceDegree3And4, sep = "" )

# average source degree (person type 3 only)
testAverageSourceDegreeOnly3 <- calcSourceMeanDegree( dataFrameIN = testDataDF, includeBothIN = FALSE )
paste( "average source degree (only 3) = ", testAverageSourceDegreeOnly3, sep = "" )


### `test` - More metrics

- Back to [Table of Contents](#Table-of-Contents)

Now that we have the data in statnet object, run the code in the following for more in-depth information:

* `context_text/R/sna/statnet/sna-statnet-network-stats.r`

In [ ]:
# Links:
# - manual (PDF): http://cran.r-project.org/web/packages/sna/sna.pdf
# - good notes: http://www.shizukalab.com/toolkits/sna/node-level-calculations

# Also, be advised that statnet and igraph don't really play nice together.
#    If you'll be using both, best idea is to have a workspace for each.

#==============================================================================#
# statnet
#==============================================================================#

# make sure you've loaded the statnet library (includes sna)
# install.packages( "statnet" )
#library( statnet )

#==============================================================================#
# NODE level
#==============================================================================#

# what is the standard deviation of the degrees?
testDegreeSd <- sd( testDegreeVector )
paste( "degree SD = ", testDegreeSd, sep = "" )

# what is the variance of the degrees?
testDegreeVar <- var( testDegreeVector )
paste( "degree variance = ", testDegreeVar, sep = "" )

# what is the max value among the degrees?
testDegreeMax <- max( testDegreeVector )
paste( "degree max = ", testDegreeMax, sep = "" )

# calculate and plot degree distributions
testDegreeFrequenciesTable <- table( testDegreeVector )
paste( "degree frequencies = ", testDegreeFrequenciesTable, sep = "" )
testDegreeFrequenciesTable

# node-level undirected betweenness
testBetweenness <- sna::betweenness( testNetworkStatnet, gmode = "graph", cmode = "undirected" )

#paste( "betweenness = ", testBetweenness, sep = "" )
# associate with each node as a node attribute.
#    (%v% is a shortcut for the get.vertex.attribute command)
testNetworkStatnet %v% "betweenness" <- testBetweenness

# also add degree vector to original data frame
testDataDF$betweenness <- testBetweenness

#==============================================================================#
# NETWORK level
#==============================================================================#

# graph-level degree centrality
testDegreeCentrality <- sna::centralization( testNetworkStatnet, sna::degree, mode = "graph" )
paste( "degree centrality = ", testDegreeCentrality, sep = "" )

# graph-level betweenness centrality
testBetweennessCentrality <- sna::centralization( testNetworkStatnet, sna::betweenness, mode = "graph", cmode = "undirected" )
paste( "betweenness centrality = ", testBetweennessCentrality, sep = "" )

# graph-level connectedness
testConnectedness <- sna::connectedness( testNetworkStatnet )
paste( "connectedness = ", testConnectedness, sep = "" )

# graph-level transitivity
testTransitivity <- sna::gtrans( testNetworkStatnet, mode = "graph" )
paste( "transitivity = ", testTransitivity, sep = "" )

# graph-level density
testDensity <- sna::gden( testNetworkStatnet, mode = "graph" )
paste( "density = ", testDensity, sep = "" )


### `test` - create node attribute DataFrame

- Back to [Table of Contents](#Table-of-Contents)

If you want to just work with the traits of the nodes/vertexes, you can combine the attribute vectors into a data frame.

In [ ]:
#==============================================================================#
# output attributes to data frame
#==============================================================================#

# if you want to just work with the traits of the nodes/vertexes, you can
#    combine the attribute vectors into a data frame.

# first, output network object to see what attributes you have
testNetworkStatnet

# then, combine them into a data frame.
testNodeAttrDF <- data.frame( id = testNetworkStatnet %v% "vertex.names",
                                     person_id = testNetworkStatnet %v% "person_id",
                                     person_type = testNetworkStatnet %v% "person_type",
                                     degree = testNetworkStatnet %v% "degree",
                                     betweenness = testNetworkStatnet %v% "betweenness" )


## Appendix - QAP network correlation

### Load matrix 1 - before

In [ ]:
# initialize variables
beforeDataFolder <- data_directory
beforeDataFile <- "grp_6mos_by_month_2006-06-09_pre-20220604-044842.tab"
beforeDataPath <- paste( beforeDataFolder, "/", beforeDataFile, sep = "" )

In [ ]:
beforeDataPath

Load the data file into memory

In [ ]:
# tab-delimited:
beforeDataDF <- read.delim( beforeDataPath, header = TRUE, row.names = 1, check.names = FALSE )

In [ ]:
# get count of rows...
beforeRowCount <- nrow( beforeDataDF )
paste( "before row count = ", beforeRowCount, sep = "" )

# ...and columns
beforeColumnCount <- ncol( beforeDataDF )
paste( "before column count = ", beforeColumnCount, sep = "" )

Get just the tie rows and columns for initializing network libraries.

In [ ]:
# the below syntax returns only as many columns as there are rows, so
#     omitting any trait columns that lie in columns on the right side
#     of the file.
beforeNetworkDF <- beforeDataDF[ , 1 : beforeRowCount ]
#str( beforeNetworkDF )

In [ ]:
# convert to a matrix
beforeNetworkMatrix <- as.matrix( beforeNetworkDF )
# str( beforeNetworkMatrix )

### Load matrix 2 - after

In [ ]:
# initialize variables
afterDataFolder <- data_directory
afterDataFile <- "grp_6mos_by_month_2006-06-09_post-20220604-045115.tab"
afterDataPath <- paste( afterDataFolder, "/", afterDataFile, sep = "" )

In [ ]:
afterDataPath

Load the data file into memory

In [ ]:
# tab-delimited:
afterDataDF <- read.delim( afterDataPath, header = TRUE, row.names = 1, check.names = FALSE )

In [ ]:
# get count of rows...
afterRowCount <- nrow(afterDataDF )
paste( "after row count = ", afterRowCount, sep = "" )

# ...and columns
afterColumnCount <- ncol( afterDataDF )
paste( "after column count = ", afterColumnCount, sep = "" )

Get just the tie rows and columns for initializing network libraries.

In [ ]:
# the below syntax returns only as many columns as there are rows, so
#     omitting any trait columns that lie in columns on the right side
#     of the file.
afterNetworkDF <- afterDataDF[ , 1 : afterRowCount ]
#str( afterNetworkDF )

In [ ]:
# convert to a matrix
afterNetworkMatrix <- as.matrix( afterNetworkDF )
# str( afterNetworkMatrix )

### run QAP correlation

In [ ]:
#outputPrefix <- "years_before_to_after_layoffs"
outputPrefix <- "six_month_test"

In [ ]:
beforeToAfterLayoffOutput <- compareMatricesQAP( beforeNetworkMatrix, afterNetworkMatrix, outputPrefix, doQapIN = FALSE )
#beforeToAfterLayoffOutput <- compareMatricesQAP( beforeNetworkMatrix, afterNetworkMatrix, outputPrefix, repsIN = 10 )

In [ ]:
# also output plots of distributions of QAP values?
displayCompareMatricesQAPOutput( beforeToAfterLayoffOutput, outputPrefix, TRUE )